### קוד מבוא

#### ספריות

In [1]:
import os
import sys
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

#### העלאת משתנים להרצת הקוד

In [16]:
path = os.getcwd()

df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

future_folder = os.path.dirname(software_data_folder_location)

create_forecast_basic_software_location = os.path.dirname(future_folder)

sys.path.append(future_folder)

'C:\\Users\\dpere\\Documents\\JTMT\\forecast_git\\create_forecast_basic'

### פונקציות גלובליות

In [4]:
from global_functions import up_load_shp, up_load_df, split_index_by_taz, change_cols_names

#### קיבולת ברמת יישוב (חלקי) ביו"ש

##### העלאת טבלה

In [6]:
path=r'{}\background_files'.format(software_data_folder_location)
kibolt_yosh_some_towns=up_load_df(path,'230628_Jewish_kibolt_some_towns_in_yosh_till_2050')

In [7]:
col=[ 'Taz_num', 'add_aprt_2020__2025',
 'add_aprt_2025__2030',
 'add_aprt_2030__2035',
 'add_aprt_2035__2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050']

kibolt_yosh_some_towns_sum_by_taz=kibolt_yosh_some_towns.pivot_table(index='Taz_num',aggfunc='sum').reset_index()[col]

kibolt_yosh_some_towns_sum_by_taz['kibolet_type']='Shrinkable'

In [8]:
old_col=[ 'Taz_num','add_aprt_2020__2025',
 'add_aprt_2025__2030',
 'add_aprt_2030__2035',
 'add_aprt_2035__2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050','kibolet_type']

new_col=[ 'Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_type']

kibolt_yosh_some_towns_sum_by_taz=change_cols_names(kibolt_yosh_some_towns_sum_by_taz, old_col, new_col)

kibolt_yosh_some_towns_sum_by_taz=kibolt_yosh_some_towns_sum_by_taz.melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

kibolt_yosh_some_towns_sum_by_taz['source']='by_town'

#### מעקב אחרי מקורות קיבולת לכל TAZ

##### העלאת טבלה

In [9]:
# taz=up_load_shp(r'{}\background_files\TAZ_V4_230518_Published.shp'.format(software_data_folder_location))
taz=up_load_shp(r'{}\background_files\TAZ_V4_240404_with_geo_info.shp'.format(create_forecast_basic_software_location))

In [28]:
taz.loc[taz['Taz_num'].isin(list(kibolt_yosh_some_towns_sum_by_taz.index)),'by_town']=1

#### קיבולת ברמת אזור תנועה (חלקי)

##### העלאת טבלה

In [11]:
Jewish_kibolt_some_by_taz=up_load_shp(r'{}\background_files\kibolt_jew_by_some_taz_230628.shp'.format(software_data_folder_location))

##### שינוי שמות של העמודות

In [12]:
old_col=[ 'Taz_num','add_aprt_2',
 'add_aprt_3',
 'add_aprt_4',
 'add_aprt_5',
 'add_aprt_6',
 'add_aprt_7','type_kibol']

new_col=[ 'Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_type']

Jewish_kibolt_some_by_taz=change_cols_names(Jewish_kibolt_some_by_taz, old_col, new_col)

In [13]:
Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['kibolet_type']==1,'kibolet_type']='Shrinkable'

Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['kibolet_type']==2,'kibolet_type']='left_the_station'

Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['kibolet_type']==3,'kibolet_type']='Depends_on_policy'

Jewish_kibolt_some_by_taz=Jewish_kibolt_some_by_taz.melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

Jewish_kibolt_some_by_taz['source']='by_taz'

C:\Users\dpere\AppData\Local\Temp\ipykernel_31344\2960630682.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Shrinkable' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Jewish_kibolt_some_by_taz.loc[Jewish_kibolt_some_by_taz['kibolet_type']==1,'kibolet_type']='Shrinkable'


##### מעקב אחרי מקורות קיבולת לכל TAZ

In [14]:
taz.loc[taz['Taz_num'].isin(list(Jewish_kibolt_some_by_taz.Taz_num)),'by_taz']=1

#### קיבולת ברמת תכנית עם אזורי תנועה בלבד (חלקי)

##### העלאת טבלה

In [15]:
Jewish_kibolt_some_by_plan_only_with_taz=up_load_shp(r'{}\background_files\Jewish_kibolt_some_by_plan_with_taz_num.shp'.format(software_data_folder_location))

In [16]:
Jewish_kibolt_some_by_plan_only_with_taz=Jewish_kibolt_some_by_plan_only_with_taz[['Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_ty']].rename(columns={'kibolet_ty':'kibolet_type'})

Jewish_kibolt_some_by_plan_only_with_taz=Jewish_kibolt_some_by_plan_only_with_taz.melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

Jewish_kibolt_some_by_plan_only_with_taz['source']='plan_only_with_taz'

##### מעקב אחרי מקורות קיבולת לכל TAZ

In [17]:
taz.loc[taz['Taz_num'].isin(list(Jewish_kibolt_some_by_plan_only_with_taz.Taz_num)),'plan_only_with_taz']=1

#### קיבולת ברמת תכנית  (חלקי)

##### העלאת טבלה

In [18]:
df=up_load_df(r'{}\background_files'.format(software_data_folder_location),'index_for_id_polygons_230717')

c:\Users\dpere\miniconda3\envs\creat_forecast_ad_hoc_env\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


##### העלאת שכבה

In [19]:
shp=up_load_shp(r'{}\background_files\230628_id_polygons_for_index.shp'.format(software_data_folder_location))

In [20]:
shp=up_load_shp(r'{}\background_files\230628_id_polygons_for_index.shp'.format(software_data_folder_location))


index=shp.merge(df,on='id',how='outer')

#לחלק את התכניות לפי אזורי תנועה

index=split_index_by_taz(index,taz,0.25,['add_aprt'])

#לייצר קיבולת כל חומש

years=[ '2025', '2030', '2035', '2040', '2045', '2050']

index[years]=index[years].apply(lambda x: x * index['risk_factor']*index['add_aprt'])

col=['Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050','kibolet_type']

Jewish_kibolt_some_by_plan=index[col].fillna(0).melt(id_vars=['Taz_num', 'kibolet_type'], var_name='year', value_name='add_aprt').query('add_aprt!=0')

Jewish_kibolt_some_by_plan['source']='by_plan'

c:\Users\dpere\miniconda3\envs\creat_forecast_ad_hoc_env\Lib\site-packages\geopandas\geodataframe.py:2469: UserWarning: `keep_geom_type=True` in overlay resulted in 13 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


### מכאן  צריך לחבר ביחד את כל הטבלאות

In [21]:
kibolt_jew=pd.concat([Jewish_kibolt_some_by_plan,Jewish_kibolt_some_by_plan_only_with_taz,kibolt_yosh_some_towns_sum_by_taz,Jewish_kibolt_some_by_taz],axis=0)

kibolt_jew['year']=kibolt_jew['year'].astype(str)

In [22]:
# תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

##### ייצוא טבלה

In [23]:
kibolt_jew.to_excel(r'{}\Intermediates\{}_kibolt_jew_2050_by_type.xlsx'.format(software_data_folder_location,file_date),index=False)

### ייצוא לקלט של מכונת התפלגות גילים תרחיש צתאל

In [24]:
kibolt_jew_jtmt=kibolt_jew.pivot_table(index='Taz_num',columns='year',values='add_aprt',aggfunc='sum').fillna(0).reset_index()

old_col=[ 'Taz_num','2025',
 '2030',
 '2035',
 '2040',
 '2045',
 '2050']

new_col=[ 'Taz_num','add_aprt_2020_2025',
 'add_aprt_2025_2030',
 'add_aprt_2030_2035',
 'add_aprt_2035_2040',
 'add_aprt_2040_2045',
 'add_aprt_2045_2050']

kibolt_jew_jtmt=change_cols_names(kibolt_jew_jtmt, old_col, new_col)

kibolt_jew_jtmt.to_excel(r'{}\Intermediates\{}_kibolt_jew_till_2050.xlsx'.format(software_data_folder_location,file_date),index=False)